In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model,save_model,Model
from tensorflow.keras.layers import Dense,Input,Flatten
import tensorflow as tf
physical_devices = tf.config.list_physical_devices("GPU")
from sklearn.preprocessing import MinMaxScaler

# my_seed = 7
# tf.compat.v1.set_random_seed(my_seed)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0],True)
    assert tf.config.experimental.get_memory_growth(physical_devices[0])
except:
    pass

In [2]:
data = pd.read_csv('data/da1.csv',index_col="time")
data = data.iloc[:,[0,7]]
data

,number,category
time,,
2002/9/23,206,3
2002/9/24,192,3
2002/9/25,245,3
2002/9/26,209,3
2002/9/27,225,3
...,...,...
2004/5/10,108,2
2004/5/11,110,2
2004/5/12,114,2


In [3]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(data['number'].values.reshape(-1,1))

In [4]:
lookback = 7

In [5]:
def create_data(dataset,data,lookback):
    dataX,dataY = [],[]
    for i in range(len(dataset) - lookback):
        dataX.append(dataset[i:i+lookback,[0]])
        dataY.append(data.iloc[i+lookback,[1]])
    return np.array(dataX),np.array(dataY)

In [6]:
dataX,dataY = create_data(dataset,data,lookback)

In [7]:
dataX[0],dataX[1]

(array([[0.67672414],
        [0.61637931],
        [0.84482759],
        [0.68965517],
        [0.75862069],
        [0.45258621],
        [0.43103448]]),
 array([[0.61637931],
        [0.84482759],
        [0.68965517],
        [0.75862069],
        [0.45258621],
        [0.43103448],
        [0.75      ]]))

In [8]:
dataX = dataX.reshape(dataX.shape[0],dataX.shape[1])

In [9]:
dataX.shape,dataY.shape

((593, 7), (593, 1))

In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [11]:
one_hots = to_categorical(dataY-1)

In [12]:
one_hots.shape

(593, 3)

In [13]:
trainX,testX,trainY,testY = train_test_split(dataX,one_hots,test_size=0.20,random_state=1)
# trainX,testX,trainY,testY = train_test_split(dataX,one_hots,test_size=0.20)

In [14]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [15]:
# 两层比较好     units = 32
inputs = Input(shape=(trainX.shape[1],))
x = Dense(16, activation='relu')(inputs)
x = Dense(16, activation='relu')(x)
x = Dense(16, activation='relu')(x)
outputs = Dense(3, activation = 'softmax')(x)

In [16]:
model = Model(inputs = inputs,outputs = outputs)

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7)]               0         
                                                                 
 dense (Dense)               (None, 16)                128       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 3)                 51        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model.fit(trainX,trainY,epochs=100, batch_size=128, verbose=2,validation_split=0.2,callbacks = [callback])
# history = model.fit(trainX,trainY,epochs=150, batch_size=12, verbose=2,validation_split=0.2)

Epoch 1/100
3/3 - 0s - loss: 1.0969 - accuracy: 0.3193 - val_loss: 1.0949 - val_accuracy: 0.3158 - 355ms/epoch - 118ms/step
Epoch 2/100
3/3 - 0s - loss: 1.0918 - accuracy: 0.3245 - val_loss: 1.0897 - val_accuracy: 0.3158 - 16ms/epoch - 5ms/step
Epoch 3/100
3/3 - 0s - loss: 1.0877 - accuracy: 0.3245 - val_loss: 1.0844 - val_accuracy: 0.3158 - 15ms/epoch - 5ms/step
Epoch 4/100
3/3 - 0s - loss: 1.0825 - accuracy: 0.3245 - val_loss: 1.0793 - val_accuracy: 0.3263 - 16ms/epoch - 5ms/step
Epoch 5/100
3/3 - 0s - loss: 1.0780 - accuracy: 0.3245 - val_loss: 1.0749 - val_accuracy: 0.3368 - 16ms/epoch - 5ms/step
Epoch 6/100
3/3 - 0s - loss: 1.0734 - accuracy: 0.3272 - val_loss: 1.0709 - val_accuracy: 0.3368 - 15ms/epoch - 5ms/step
Epoch 7/100
3/3 - 0s - loss: 1.0690 - accuracy: 0.3430 - val_loss: 1.0672 - val_accuracy: 0.3474 - 17ms/epoch - 6ms/step
Epoch 8/100
3/3 - 0s - loss: 1.0646 - accuracy: 0.3588 - val_loss: 1.0632 - val_accuracy: 0.3368 - 19ms/epoch - 6ms/step
Epoch 9/100
3/3 - 0s - loss: 

Epoch 69/100
3/3 - 0s - loss: 0.6792 - accuracy: 0.6887 - val_loss: 0.6654 - val_accuracy: 0.7158 - 17ms/epoch - 6ms/step
Epoch 70/100
3/3 - 0s - loss: 0.6776 - accuracy: 0.6860 - val_loss: 0.6525 - val_accuracy: 0.6632 - 18ms/epoch - 6ms/step
Epoch 71/100
3/3 - 0s - loss: 0.6768 - accuracy: 0.6781 - val_loss: 0.6538 - val_accuracy: 0.6737 - 14ms/epoch - 5ms/step
Epoch 72/100
3/3 - 0s - loss: 0.6771 - accuracy: 0.6781 - val_loss: 0.6650 - val_accuracy: 0.7158 - 15ms/epoch - 5ms/step
Epoch 73/100
3/3 - 0s - loss: 0.6737 - accuracy: 0.6860 - val_loss: 0.6589 - val_accuracy: 0.7263 - 15ms/epoch - 5ms/step
Epoch 74/100
3/3 - 0s - loss: 0.6725 - accuracy: 0.6992 - val_loss: 0.6525 - val_accuracy: 0.6737 - 15ms/epoch - 5ms/step
Epoch 75/100
3/3 - 0s - loss: 0.6711 - accuracy: 0.6939 - val_loss: 0.6519 - val_accuracy: 0.6947 - 15ms/epoch - 5ms/step
Epoch 76/100
3/3 - 0s - loss: 0.6707 - accuracy: 0.6966 - val_loss: 0.6535 - val_accuracy: 0.7053 - 15ms/epoch - 5ms/step
Epoch 77/100
3/3 - 0s - 

In [20]:
accuracy = model.evaluate(testX,testY, batch_size=128, verbose=2)

1/1 - 0s - loss: 0.7199 - accuracy: 0.6555 - 11ms/epoch - 11ms/step


In [21]:
testPredict = model.predict(testX)

In [22]:
# model = load_model('model/mlp2.h5')
# testPredict = model.predict(testX)

In [23]:
testPredict

array([[3.67950410e-01, 5.51159024e-01, 8.08905363e-02],
       [5.27337968e-01, 4.18094903e-01, 5.45671023e-02],
       [1.93653833e-02, 3.00392747e-01, 6.80241883e-01],
       [1.88135996e-01, 5.25973558e-01, 2.85890371e-01],
       [7.71335959e-01, 2.13988006e-01, 1.46760382e-02],
       [7.58636534e-01, 2.30624750e-01, 1.07388049e-02],
       [4.32538956e-01, 4.96150672e-01, 7.13104457e-02],
       [7.85228133e-01, 2.06037477e-01, 8.73444229e-03],
       [4.37447846e-01, 4.84172881e-01, 7.83792734e-02],
       [8.80488660e-05, 2.76187640e-02, 9.72293139e-01],
       [8.18988144e-01, 1.73629686e-01, 7.38220150e-03],
       [9.77645814e-02, 5.05329430e-01, 3.96906018e-01],
       [3.02209944e-01, 5.47604024e-01, 1.50186047e-01],
       [2.32587104e-06, 3.86444107e-03, 9.96133208e-01],
       [9.44671929e-02, 5.43112218e-01, 3.62420589e-01],
       [1.57398850e-01, 5.34414649e-01, 3.08186501e-01],
       [5.02838306e-02, 5.07370770e-01, 4.42345351e-01],
       [5.77920258e-01, 3.90710

In [24]:
testPredict.shape

(119, 3)

In [25]:
y_pred = np.argmax(testPredict, axis=1)

In [26]:
y_pred

array([1, 0, 2, 1, 0, 0, 1, 0, 1, 2, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 2,
       1, 2, 2, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 2, 1, 2, 0, 2, 2, 2, 0, 1,
       0, 1, 2, 0, 1, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 1,
       1, 0, 0, 0, 0, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 2, 0, 2, 0, 0, 2, 2,
       1, 0, 0, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 2, 1, 0, 2, 1, 1, 0, 0], dtype=int64)

In [27]:
y_real = np.argmax(testY, axis=1)

In [28]:
y_real

array([0, 0, 1, 2, 0, 1, 1, 0, 1, 2, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 2,
       0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 2, 2, 1, 0, 1, 2, 1, 2, 1,
       0, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 0, 1,
       0, 0, 1, 0, 0, 1, 2, 2, 1, 0, 0, 2, 0, 1, 2, 2, 1, 0, 0, 0, 2, 1,
       1, 0, 0, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 2, 2, 2, 0,
       1, 1, 1, 0, 2, 2, 0, 1, 1], dtype=int64)

In [29]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_real,y_pred)
# kappa = cohen_kappa_score(dataY[-30:],y_pred)
#(label除非是你想计算其中的分类子集的kappa系数，否则不需要设置)
kappa

0.47548914211997406

In [30]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(y_real,y_pred))
# print(classification_report(dataY[-30:],y_pred))

              precision    recall  f1-score   support

           0       0.71      0.76      0.73        45
           1       0.60      0.53      0.56        45
           2       0.65      0.69      0.67        29

    accuracy                           0.66       119
   macro avg       0.65      0.66      0.65       119
weighted avg       0.65      0.66      0.65       119



In [32]:
# model.save('mlp11.h5')

In [33]:
classification_report(y_real,y_pred,output_dict=True)

{'0': {'precision': 0.7083333333333334,
  'recall': 0.7555555555555555,
  'f1-score': 0.7311827956989247,
  'support': 45},
 '1': {'precision': 0.6,
  'recall': 0.5333333333333333,
  'f1-score': 0.5647058823529412,
  'support': 45},
 '2': {'precision': 0.6451612903225806,
  'recall': 0.6896551724137931,
  'f1-score': 0.6666666666666667,
  'support': 29},
 'accuracy': 0.6554621848739496,
 'macro avg': {'precision': 0.6511648745519714,
  'recall': 0.659514687100894,
  'f1-score': 0.6541851149061776,
  'support': 119},
 'weighted avg': {'precision': 0.6519720791542424,
  'recall': 0.6554621848739496,
  'f1-score': 0.6525069230728345,
  'support': 119}}

In [37]:
result = classification_report(y_real,y_pred,output_dict=True)
print(result['macro avg'])
print(result['macro avg']['precision'])
print(result['macro avg']['recall'])
print(result['macro avg']['f1-score'])

{'precision': 0.6511648745519714, 'recall': 0.659514687100894, 'f1-score': 0.6541851149061776, 'support': 119}
0.6511648745519714
0.659514687100894
0.6541851149061776
